<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [96]:
!mamba install pandas==1.3.3 -y
!mamba install requests==2.26.0 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.27.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pandas==1.3.3']

[+] 0.0s
opt/ibm/custom-channels/meta-wscloud/linux-64 ━━━━━━━━━━━╸━━━━━━━━   0.

Extracting   (2) ━━━━━━━━━╸━━━━━━━━━━━━━       0 requests                   0.6s[+] 0.8s
Extracting   (2) ━━━━━━━━━━╸━━━━━━━━━━━━       0 requests                   0.7s[+] 0.9s
Extracting   (2) ━━━━━━━━━━━╸━━━━━━━━━━━       0 ibm-wsrt-py310t2-def       0.8s[+] 1.0s
Extracting   (2) ━━━━━━━━━━━━╸━━━━━━━━━━       0 ibm-wsrt-py310t2-def       0.9s[+] 1.1s
Extracting   (2) ━━━━━━━━━━━━━╸━━━━━━━━━       0 ibm-wsrt-py310t2-def       1.0s[+] 1.2s
Extracting   (2) ━━━━━━━━━━━━━━╸━━━━━━━━       0 ibm-wsrt-py310t2-def       1.1s[+] 1.3s
Extracting   (2) ━━━━━━━━━━━━━━━╸━━━━━━━       0 requests                   1.2s[+] 1.4s
Extracting   (1) ━━━━━━━━━━╸━━━━━━━━━━━━       1 requests                   1.3s[+] 1.5s
Extracting   (1) ━━━━━━━━━━╸━━━━━━━━━━━━       1 requests                   1.4s[+] 1.6s
Extracting   (1) ━━━━━━━━━━╸━━━━━━━━━━━━       1 requests                   1.5s[+] 1.7s
Extracting       ━━━━━━━━━━━━━━━━━━━━━━━       2                            1.6sPreparing transaction: done
Ve

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [97]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-04-30 13:06:41--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.1’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-04-30 13:06:42 (53.5 MB/s) - ‘bank_market_cap_1.json.1’ saved [2815/2815]

--2023-04-30 13:06:44--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%2

## Extract


### JSON Extract Function

This function will extract JSON files.


In [98]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [99]:
columns=['Name','Market Cap (US$ Billion)']

In [100]:
def extract():
    extracted_data = pd.DataFrame(columns=columns)
    for json_file in glob.glob("*.json"):
        extracted_data = extract_from_json(json_file)
                            
    return extracted_data
    
    

<b> 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.
Question

In [101]:
# Write your code here
df = pd.read_csv(r'exchange_rates.csv')
df = pd.DataFrame(df)
print(df)
exchange_rate=df.loc[9, 'Rates']
print(exchange_rate) 





   Unnamed: 0         Rates
0         AUD      1.297088
1         BGN      1.608653
2         BRL      5.409196
3         CAD      1.271426
4         CHF      0.886083
5         CNY      6.483139
6         CZK     21.510117
7         DKK      6.119757
8         EUR      0.822504
9         GBP      0.732398
10        HKD      7.752509
11        HRK      6.222652
12        HUF    294.135549
13        IDR  14097.902616
14        ILS      3.281132
15        INR     73.011186
16        ISK    129.133081
17        JPY    103.791742
18        KRW   1105.839776
19        MXN     19.932966
20        MYR      4.043017
21        NOK      8.497121
22        NZD      1.393733
23        PHP     48.070406
24        PLN      3.732933
25        RON      4.008883
26        RUB     74.930827
27        SEK      8.292071
28        SGD      1.328261
29        THB     30.009870
30        TRY      7.418572
31        USD      1.000000
32        ZAR     15.118441
0.7323984208000001


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [102]:
def transform(data):
    # Write your code here
    data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)'] * exchange_rate, 3)
    data = data.rename(columns = {"Market Cap (US$ Billion)":"Market Cap (GBP£ Billion)"}, inplace=False)
    return data
    

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [103]:
targetfile = "bank_market_cap_gbp.csv"
def load(targetfile, datatoload):
    # Write your code here
    datatoload.to_csv(targetfile, index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [104]:
def log(message):
    # Write your code here
    timestamp_format = '%Y-%h-%d-%H%M%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp+','+message+'\n')
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [105]:
# Write your code here
log("ETL Job Started")
log("Extract Phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [106]:
# Call the function here
extract_data = extract()
extract_data
# Print the rows here
print(extract_data.head())

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399


Log the data as <code>"Extract phase Ended"</code>


In [107]:
log("Extract Phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [108]:
log("Transform Phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [109]:
transformed_data = transform(extract_data)
transformed_data
print(transformed_data.head(5))


                                      Name  Market Cap (GBP£ Billion)
0                           JPMorgan Chase                    286.319
1  Industrial and Commercial Bank of China                    252.834
2                          Bank of America                    238.272
3                              Wells Fargo                    225.588
4                  China Construction Bank                    188.519


Log your data <code>"Transform phase Ended"</code>


In [110]:
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [111]:
log("Load phase Started")

Call the load function


In [112]:
load(targetfile,transformed_data)

Log the following `"Load phase Ended"`.


In [113]:
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
